In [12]:
"""
#基于python 3.6.5 tensorflow 1.8.0
#tf.train.Example主要用在将数据处理成二进制方面，一般是为了提升IO效率和方便管理数据

tf.train.BytesList等
tf.train.Feature
tf.train.Features
tf.train.Example
"""

import struct
import tensorflow as tf

def read_text_file(text_file):
    """
    21
    This is a test data file.
    We will convert this text file to bin file.
    1
    2
    3
    """
    lines = []
    with open(text_file, "r") as f:
        for line in f:
            lines.append(line.strip())
    return lines

def text_to_binary(in_file, out_file):
    inputs = read_text_file(in_file)
    """
    #格式化原始数据可以使用tf.train.BytesList tf.train.Int64List tf.train.FloatList三个类。
    #这三个类都有实例属性value用于我们将值传进去
    #一般tf.train.Int64List tf.train.FloatList对应处理整数和浮点数，tf.train.BytesList用于处理其他类型的数据
    """
    data_id = tf.train.Int64List(value=[int(inputs[0])])
    data    = tf.train.BytesList(value=[bytes(' '.join(inputs[1:]), encoding='utf-8')])
    print(data_id)
    print(data)
    
    """
    tf.train.Feature有三个属性为tf.train.bytes_list tf.train.float_list tf.train.int64_list
    """
    tf.train.Feature(int64_list=data_id)
    tf.train.Feature(bytes_list=data)

    feature_dict = {
        "data_id": tf.train.Feature(int64_list=data_id),
        "data": tf.train.Feature(bytes_list=data)
    }
    
    """
    从名字来看，我们应该能猜出tf.train.Features是tf.train.Feature的复数
    事实上tf.train.Features有属性为feature，这个属性的一般设置方法是传入一个字典，字典的key是字符串（feature名）
    而值是tf.train.Feature对象
    """
    features = tf.train.Features(feature=feature_dict)
    """
    tf.train.Example还有一个方法SerializeToString()需要说一下，这个方法的作用是把tf.train.Example对象序列化为字符串
    因为我们写入文件的时候不能直接处理对象，需要将其转化为字符串才能处理
    """
    example = tf.train.Example(features=features)
    example_str = example.SerializeToString()
    
    str_len = len(example_str)
    
    print(example_str)

    with open(out_file, 'wb') as writer:
        writer.write(struct.pack('H', str_len))
        writer.write(struct.pack('%ds' % str_len, example_str))

if __name__ == '__main__':
    text_to_binary('data/train_example.txt', 'data/train_example.bin')
    tf.ListDirectory("data")

value: 21

value: "This is a test data file. We will convert this text file to bin file. 1 2 3"

b'\nk\n\x10\n\x07data_id\x12\x05\x1a\x03\n\x01\x15\nW\n\x04data\x12O\nM\nKThis is a test data file. We will convert this text file to bin file. 1 2 3'


AttributeError: module 'tensorflow' has no attribute 'ListDirectory'